## Introduction to Massive Data Analysis 
### HW4 - Finding Similar Articles
106070038 杜葳葳

匯入 SparkConf、SparkContext、Pyspark模組<br>
設定環境變數（解決版本問題）

In [81]:
# import package
from pyspark import SparkConf,SparkContext
import pyspark
from numpy import array
import csv

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools

# set environment variables
import os
os.environ["PYSPARK_PYTHON"]="/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8"
os.environ['PYSPARK_DRIVER_PYTHON'] = '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8'

建立 SparkConf 物件<br>
設為 local 模式、名稱設為 "hw4"

In [287]:
sc.stop()

In [288]:
conf = SparkConf().setMaster("local").setAppName("hw4")
sc = SparkContext(conf=conf)
sc

<SparkContext master=local appName=hw4>

### 讀檔＆前處理
去掉”-“、”.“、”\n“、","、”'“、”\“<br>
全部改成小寫<br>

In [289]:
## 資料夾目錄 
path = "./Downloads/athletics"
## 得到資料夾下的所有檔名稱
files= os.listdir(path) 
## 排序
files.sort(key=lambda x: int(x.split('.')[0]))
print(files)
dataset = [] 
count = 0
for file in files:
    count = count + 1
    f = open(path+"/"+file)
    data = f.read()
    data = data.replace("-"," ").replace(".","").replace("\n"," ").replace(",","").replace("'","").replace("\"","")
    data = data.lower()
    data = data.split(" ")
    while "" in data:
        data.remove("")
    dataset.append(data)
print(count)

['001.txt', '002.txt', '003.txt', '004.txt', '005.txt', '006.txt', '007.txt', '008.txt', '009.txt', '010.txt', '011.txt', '012.txt', '013.txt', '014.txt', '015.txt', '016.txt', '017.txt', '018.txt', '019.txt', '020.txt', '021.txt', '022.txt', '023.txt', '024.txt', '025.txt', '026.txt', '027.txt', '028.txt', '029.txt', '030.txt', '031.txt', '032.txt', '033.txt', '034.txt', '035.txt', '036.txt', '037.txt', '038.txt', '039.txt', '040.txt', '041.txt', '042.txt', '043.txt', '044.txt', '045.txt', '046.txt', '047.txt', '048.txt', '049.txt', '050.txt', '051.txt', '052.txt', '053.txt', '054.txt', '055.txt', '056.txt', '057.txt', '058.txt', '059.txt', '060.txt', '061.txt', '062.txt', '063.txt', '064.txt', '065.txt', '066.txt', '067.txt', '068.txt', '069.txt', '070.txt', '071.txt', '072.txt', '073.txt', '074.txt', '075.txt', '076.txt', '077.txt', '078.txt', '079.txt', '080.txt', '081.txt', '082.txt', '083.txt', '084.txt', '085.txt', '086.txt', '087.txt', '088.txt', '089.txt', '090.txt', '091.txt'

In [290]:
## 建立RDD,將每個doc加上index
data = sc.parallelize(dataset).zipWithIndex().map(lambda x: (x[1]+1,x[0]))
#data.take(1)

#### 資料型別
* List 用中括號 [] 表示:<br>
    有順序，可改變內容的序列
* Tuple 用小括號 () 表示:<br>
    有順序，不可改變內容的序列
* Set 用大括號 {} 表示:<br>
    資料的集合，沒有順序，沒有重複，且為可改變內容
* Dict 用大括號 {} 表示:<br>
    沒有順序，沒有重複，且為可改變內容的多個鍵

### 1. Shingling
Convert Doc to Sets<br>
k=3,將每三個words放入一個tuple<br>
並將每個doc中的所有tuple放入同個set中<br>
因為set內的元素不可重複,重複的只會被算到一次,剛好符合shingling中的要求<br>

In [291]:
def three_shingle(data):
    text = data[1]
    total_shingle = len(data[1])-3 +1
    shingle_list = []
    for i in range(total_shingle):
        shingle_list.append([text[i],text[i+1],text[i+2]])   
        
    shingle_set = set(tuple(s) for s in shingle_list)
    return(data[0],shingle_set)
tuple_data = data.map(three_shingle)
#tuple_data.collect()

### Compressing Shingles
將shingles做hashing處理<br>
因為實際內容不重要,只在意相似度

In [292]:
def mapAndHash (data):
    maplist = []
    for i in data[1]:
        maplist.append((data[0],hash(i)))
    return maplist
shingle_data = tuple_data.flatMap(mapAndHash)
#shingle_data.collect()

### 2. Minhashing
Permuting 的計算量太大<br>
所以使用100個隨機哈希函數來模擬隨機排列轉換的效果<br>
根據哈希之後的位置建構Signature Matrix<br>
一開始將所有Signature Matrix的值都設為無限大<br>
接著尋找第r行為1,更新Signature Matrix為最小值<br>

**Universal Hashing** <br>
ha,b(x)=((ax+b)mod P)mod N<br>
a,b: random integers<br>
N: 相異shingle個數<br>
P: prime number(P>N)<br>


In [293]:
def is_prime(n):
    for i in range(2, n):
        if n % i == 0: 
            return False
    return True

def find_prime(n):
    while is_prime(n) == False:
        n = n+1
    else:
        return n

In [294]:
shingle_count = shingle_data.map(lambda x : (x[1],1)).reduceByKey(lambda x,y : x+y)
N = shingle_count.count()
P = find_prime(N)

np.random.seed(1)
a = np.random.randint(0, 100000, size = 100)
b = np.random.randint(0, 100000, size = 100)

N = 26571<br>
P = 26573

使用100個hash function產生出長度為100的array

In [295]:
hash_value = shingle_data.map(lambda x : (x[0], (x[1]*a+b)%P%N))
#hash_value.collect()

得到100個hash function分別算出來的最小值

In [296]:
def min_value(a,b):
    for i in range(100):
        if a[i] < b[i]: 
            a[i] = a[i]  
        else:
            a[i] = b[i]
    return a
min_hash = hash_value.reduceByKey(lambda x,y : min_value(x,y))
#min_hash.collect()

### 3. Locality-sensitive hashing 
b = 50, r = 2 <br>
劃分成50個band、2個行<br>
對每個band,存在一個hash能將band的每r個整數組成的列向量映射到數個大數目範圍的桶中,<br>
因為相似項比不相似項有更大機率哈希到同一個bucket,<br>
將至少有一次hash到同一個bucket的文檔看成candidate pair,<br>
檢查所有candidate pair的相似度

把原本長度為100的min_hash map到 50個不同的band

In [297]:
def hash_bucket(min_hash):
    List=[]
    for i in range(50):
        hash_value = hash((min_hash[1][2*i],min_hash[1][2*i+1]))
        List.append((min_hash[0],(i+1,hash_value)))
    return List
hash_buckets = min_hash.flatMap(hash_bucket).map(lambda x: (x[1],[x[0]])) 
#hash_buckets.take(10)

### 找到 candidate pair
將一個以上的band hash到同個bucket做為candidate pair<br>

In [298]:
candidate_pair = hash_buckets.reduceByKey(lambda x,y : x+y).values()
candidate_pair = candidate_pair.filter(lambda x: len(x)>=1)
#candidate_pair.take(10)
pairs = candidate_pair.flatMap(lambda x: list(itertools.combinations(x,2)))
pairs_count = pairs.map(lambda x: (x,1)).reduceByKey(lambda x,y : x+y)

In [299]:
#pairs_count.take(10)

### Output
top 10 indices of candidate pairs and their similarities in decreasingly order. (index1, index2) : similarity <br>
先整理一下,再計算相似度

In [300]:
C_1 = list(pairs_count.map(lambda x: x[0][0]).collect())
C_2 = list(pairs_count.map(lambda x: x[0][1]).collect())
C_12 = list(pairs_count.map(lambda x: x[0]).collect())
#C_12
columns_1 = min_hash.filter(lambda x : x[0] in C_1)
columns_2 = min_hash.filter(lambda x : x[0] in C_2)
#columns_1.take(2)

In [301]:
similar_pairs = columns_1.cartesian(columns_2).map(lambda x: ( (x[0][0],x[1][0]), (x[0][1],x[1][1]) )).filter(lambda x: x[0] in pairs_12)
#similar_pairs.collect()

### 計算 Jaccard Similarity 
相同的min-hash數量/全部的hash數量(100)<br>

In [302]:
def Jaccard_Similarity(data):
    count = 0
    for i in range(100):
        if data[1][0][i]==data[1][1][i]:
            count=count+1
    return (data[0],count/100)
similarity = similar_pairs.map(Jaccard_Similarity)

In [304]:
ans = similarity.map(lambda x : (x[1],x[0])).sortByKey(False).map(lambda x : (x[1],x[0]))
ans.collect()

[((12, 20), 1.0),
 ((52, 84), 1.0),
 ((30, 35), 0.77),
 ((47, 49), 0.75),
 ((49, 88), 0.52),
 ((23, 38), 0.48),
 ((48, 49), 0.48),
 ((14, 40), 0.45),
 ((47, 88), 0.39),
 ((47, 48), 0.36),
 ((50, 51), 0.13),
 ((13, 21), 0.12),
 ((45, 91), 0.12),
 ((81, 82), 0.08),
 ((82, 89), 0.07),
 ((34, 43), 0.05),
 ((81, 89), 0.05),
 ((9, 14), 0.04),
 ((3, 70), 0.04),
 ((14, 68), 0.04),
 ((71, 98), 0.04),
 ((1, 42), 0.03),
 ((3, 94), 0.03),
 ((14, 58), 0.03),
 ((22, 61), 0.03),
 ((33, 50), 0.03),
 ((40, 58), 0.03),
 ((40, 68), 0.03),
 ((41, 43), 0.03),
 ((33, 81), 0.03),
 ((33, 82), 0.03),
 ((37, 70), 0.03),
 ((60, 74), 0.03),
 ((13, 61), 0.02),
 ((18, 50), 0.02),
 ((33, 89), 0.02),
 ((40, 98), 0.02)]

調整格式,印出前十高的

In [305]:
top_10 = ans.take(10)
for i in top_10:
    sim = float(i[1])*100
    string = "("+str(i[0][0])+", "+str(i[0][1]) + ") : " + str(sim)+ "%"
    print(string)

(12, 20) : 100.0%
(52, 84) : 100.0%
(30, 35) : 77.0%
(47, 49) : 75.0%
(49, 88) : 52.0%
(23, 38) : 48.0%
(48, 49) : 48.0%
(14, 40) : 45.0%
(47, 88) : 39.0%
(47, 48) : 36.0%


### Final Output
(12, 20) : 100.0% <br>
(52, 84) : 100.0% <br>
(30, 35) : 77.0% <br>
(47, 49) : 75.0% <br>
(49, 88) : 52.0% <br>
(23, 38) : 48.0% <br>
(48, 49) : 48.0% <br>
(14, 40) : 45.0% <br>
(47, 88) : 39.0% <br>
(47, 48) : 36.0%